## Find NounPhrases using the Spacy library
Spacy is considered to be a very good library for POS analysis
This script is to find out the noun phrases (potential n-grams) using the spacy api. The nounphrases found will be added to the main dictionary

In [7]:
#create the instance of the spacy
from spacy.en import English
nlp = English()


Read the cases from whihc we want to find the noun phrases

In [8]:
fname="all_jim_case_large.txt"
with open(fname, 'r') as myfile:
    data=myfile.read()
data=data.split('-------BREAK--------')
cases=[case.strip() for case in data]


Read the stop word list

In [9]:
stopwordsFile = open('stopwordsss.txt', 'r')
stopwords=stopwordsFile.read()
stopwordList=stopwords.split(",")

Utility function to print a dictionary 

In [10]:
def printDict(dictToPrint):
    for key,value in dictToPrint.items():
        print(key)
        

Function to cleanup the text

In [14]:
import re
def cleanupText(str):
    str = str.strip();
    str = re.sub('/[^A-Za-z0-9 _\-\+\&\,\#]/', '', str)
    str = str.replace('"', ' ')
    str = str.replace('\"', ' ')
    str = str.replace(')', ' ')
    str = str.replace('(', ' ')
    str = str.replace('>', ' ')
    str = str.replace('@', ' ')
    str = str.replace('<', ' ')
    str = str.replace(':', ' ')
    str = str.replace('.', ' ')
    str = str.replace('[', ' ')
    str = str.replace(']', ' ')
    str = str.replace('_', ' ')
    str = str.replace(',', ' ')
    str = str.replace('#', ' ')
    str = str.replace('-', ' ')
    str = str.replace('/', ' ')
    str = str.replace('"', ' ')
    str = str.replace('\n', ' ')
    str = str.replace('~', ' ')
    str = re.sub(r'\d+', ' ', str)
    word_sent = [word for word in str.lower().split(" ") if word not in stopwordList and len(word)>1]
    if (len(word_sent) > 2):
        finalSent = ' '.join(word_sent)
    else:
        finalSent=''

    return(finalSent.strip())


Parse the cases using the spacy object

In [12]:
docs = [nlp(d) for d in cases]


Find the noun phrases from the parsed text

In [15]:
arrNounPhrase=[]
for idx, doc in enumerate(docs):
    testDoc=doc
    nounPhrases = [np.text for np in testDoc.noun_chunks]
    tokenizedNounPhrases=[]
    for tt in nounPhrases:
        finalSent=cleanupText(tt)
        if(len(finalSent)>0):
            tokenizedNounPhrases.append(finalSent)
    arrNounPhrase.append(tokenizedNounPhrases)


The arrNounPhrase is a list of list. For the further analysis we want it to be flat list so that we can do frequency count of the keys. One way to flatten the list is defining a lamba function
this has been taken from http://stackoverflow.com/questions/952914/making-a-flat-list-out-of-list-of-lists-in-python

In [16]:
flatten = lambda l: [item for sublist in l for item in sublist]
flatNounPhraseList=flatten(arrNounPhrase)

Create a dictionary with key as the ngram and value as the number of times it has appeared in the corpus

In [17]:
nounPhraseDictCount={}
for tt in flatNounPhraseList:
    finalSent=tt
    if (nounPhraseDictCount.get(finalSent)):
        nounPhraseDictCount[finalSent] += 1
    else:
        nounPhraseDictCount[finalSent] = 1
print(len(nounPhraseDictCount))


5139


Additionally we want to pick up only those keys (ngrams) that have appeared at least 10 times in the corpus

In [18]:
finalPOSList={}
for dd in nounPhraseDictCount:
    ddArr=dd.split(" ")
    if(len(ddArr)>1):
        #atelaset 10 occurence of the phrase
        if(nounPhraseDictCount[dd]>9):
            print(dd,nounPhraseDictCount[dd])
            finalPOSList[dd]=nounPhraseDictCount[dd]
        
printDict(finalPOSList)


prefilter fuel pressure 42
duct burner spread 27
bearing metal temp 62
thrust bearing temperature 19
generator bearing temperature 14
brg metal temp 17
pressure steam tag 15
combustion monitor spread 11
fw pmp mtr outboard vib 10
attemperator spray flow 15
lube oil pressure 89
bearing metal temperature 42
fuel gas temperature 15
siemens tower drive metal temperature 13
ip drum conductivity 11
gear bearing temp 13
water injection flow 20
oil drain temperature 11
compressor outlet temp range 10
exhaust gas port 12
fuel gas pressure 18
ip drum differential 25
water jacket inlet pressure 15
drive generator bearing temperature 11
fuel gas temp 21
bearing metal temps 21
ib bearing temp 16
power turbine positions 14
driver thrust bearing 10
intercooler outlet temperature 14
gen slipring bearing vib 11
sh dsh outlet temperature 10
generator bearing temperature nde 47
exhaust duct pressure 10
lp steam flow 33
nde seal outlet flow 15
rear gear bearing temp 12
motor stator temps 11
hrsg drum spec

### Perfoming statistical analysis of the nounphrases found
We want to find out what is the min and max number of occurence of each noun phrase. Which is the most occuring nounphrase 

To do this I have used numpy

In [89]:
import numpy as np

#create a new array that only has the value of the ngrams
statArr=[]
for key,value in finalPOSList.items():
    statArr.append(value)

#now convert the regular array to the numpy array. This will allow to apply
#the numpy operations
nparray=np.asarray(statArr)

#find the max value of occurence of a key (ngram)
maxVal=np.amax(nparray)

#find the min value of occurence of a key (ngram)
minVal=np.amin(nparray)
print(maxVal,"....",minVal)


89 .... 10


### using pandas to see the output in tabular form

In [93]:
import pandas as pd

def getFreqDistribution():
    #this will give the frequency count
    counts,unique = np.unique(nparray, return_counts=True)
    pp=pd.DataFrame(list(zip(counts,unique)),columns=['num of occurence','Number of such Keys'])
    print(pp.sort())

getFreqDistribution()

    num of occurence  Number of such Keys
0                 10                    9
1                 11                   13
2                 12                   14
3                 13                    8
4                 14                    5
5                 15                    8
6                 16                    5
7                 17                    4
8                 18                    2
9                 19                    5
10                20                    3
11                21                    4
12                22                    1
13                24                    3
14                25                    1
15                27                    1
16                28                    1
17                33                    1
18                39                    1
19                40                    2
20                42                    2
21                47                    1
22                49              

/Users/305015992/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(....) is deprecated, use sort_index(.....)


So we see that only 1 key has occured 89 times in the corpus. TO find out which is that key we have the following function

In [100]:
def getKeysBasedOnFrequency(freq):
    indexes=[]
    for i in range(len(nparray)):
        if (nparray[i]==freq):
            print(list(finalPOSList.keys())[i])

print("Following are the keys which have occured 89 times")                
getKeysBasedOnValue(89)

print("Following are the keys which have occured 10 times")
getKeysBasedOnValue(10)


Following are the keys which have occured 89 times
lube oil pressure
Following are the keys which have occured 10 times
stator coil vib
inlet filter dp
exhaust duct pressure
compressor outlet temp range
driver thrust bearing
agb chip detector
sh dsh outlet temperature
fw pmp mtr outboard vib
bearing oil drain temperature


Add the keys to a seperate array so that we can perform some more operations

In [19]:
ngramList=[]
#print only the keys i.e. the name of the ngram
for key,value in finalPOSList.items():
    ngramList.append(key)


### Finding additional ngrams
Now we want to find out the additional ngrams that we have obtained using the spacy. For this we have to compare with the original dictionary (assume we already have a dictionary)

In [118]:
dictFile="keywordMapping_jim.csv"
dictLines = [dictLine.rstrip('\n').split(',') for dictLine in open(dictFile)]
unigramDict={}
ngramDict={}
for ll in dictLines:
    #The ngramdict will contain normalized version both side since the 
    #assumption is when we are testing for the ngram the text is already 
    #normalized to unigrams
    if(len(ll[2].split())>1):
        str=ll[2]
        str=str.replace('"','')        
        ngramDict[str]=str
    else:
        str=ll[1]
        str=str[1:-1]
        unigramDict[str]=ll[2]
        #unigramDict[str].replace('"','')
#         ngramDict[str]=ll[2]


Utility function to check if a word exists in the dictionary

In [119]:
def my_in_array(word,dictArr):
    for idx,key in enumerate(dictArr):
        if(key==word):
            return(dictArr[key])
print(my_in_array("vlv",unigramDict))

"valve"


Now to update the dictionary. Here is the approach that I have followed
1. check which all unigrams are not in the dictionary. Add those unigrams in the dictionary
2. Normalize all the ngrams (convert pmp brg mtl to pump bearing metal)
3. Check for the presence of this normalize ngram in the dictionary
4. If not present then it needs to be added

In [120]:
#notInPrevDict will keep a track of the keys which are new
notInPrevDict=[]
for uu in ngramList:
    #uu=ngramList[1]
    words=uu.split(" ")
    for word in words:
        if(len(word)>1):
            retword=my_in_array(word,unigramDict)
            if(retword==None):
                #print(word)
                #add to the unigram dictionary
                notInPrevDict.append(word)
                unigramDict[word]=word    

In [121]:
#print(unigramDict)

In [122]:
#normalize the ngramList
normalizedNGramList=[]
for nn in ngramList:
    words = nn.split(" ")
    #print(words)
    str=''
    for word in words:
        retword=my_in_array(word,unigramDict)
        if(retword):
            retword = retword.replace('"', '')
            str += retword + " "
    #print(str)
    normalizedNGramList.append(str.strip())
#print(normalizedNGramList)


In [123]:
print(notInPrevDict)

[]


check which all ngrams are present in the dictionary

In [124]:
for nn in normalizedNGramList:
    retword=my_in_array(nn,ngramDict)
    if(retword==None):
        notInPrevDict.append(nn)


In [125]:
#these are the ones that needs to be added
print(notInPrevDict)

['duct burner fuel gas flow', 'power turbine position', 'hrsg drum spec cndc sample', 'pump ib bearing temperature', 'siemens tower drive metal temperature', 'fw pump motor outboard vibration']
